# सत्र ४ – SLM वि. LLM तुलना

लहान भाषा मॉडेल आणि मोठ्या मॉडेलमधील विलंबता आणि नमुना प्रतिसाद गुणवत्तेची तुलना Foundry Local द्वारे चालवताना करा.


## ⚡ जलद प्रारंभ

**मेमरी-ऑप्टिमाइझ केलेली सेटअप (अपडेटेड):**
1. मॉडेल्स स्वयंचलितपणे CPU प्रकार निवडतात (कोणत्याही हार्डवेअरवर कार्य करते)
2. `qwen2.5-3b` चा वापर 7B ऐवजी (सुमारे ~4GB RAM वाचते)
3. पोर्ट स्वयंचलित शोध (मॅन्युअल कॉन्फिगरेशनची गरज नाही)
4. आवश्यक एकूण RAM: ~8GB शिफारस केलेली (मॉडेल्स + OS)

**टर्मिनल सेटअप (30 सेकंद):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

मग हे नोटबुक चालवा! 🚀


### स्पष्टीकरण: अवलंबित्व स्थापना
वेळ आणि चॅट विनंत्यांसाठी आवश्यक असलेल्या किमान पॅकेजेस (`foundry-local-sdk`, `openai`, `numpy`) स्थापित करते. सुरक्षितपणे पुन्हा चालवता येते, आणि यामुळे कोणतेही दुप्पट परिणाम होत नाहीत.


# परिस्थिती
एका लहान भाषा मॉडेल (SLM) आणि एका मोठ्या मॉडेलची एका सिंगल प्रॉम्प्टवर तुलना करा, जेणेकरून खालील गोष्टींचे फायदे-तोटे स्पष्ट होतील:
- **प्रतीक्षा वेळेतील फरक** (भिंतीवरील घड्याळाच्या सेकंदांमध्ये)
- **टोकन वापर** (जर उपलब्ध असेल) थ्रूपुटसाठी एक प्रॉक्सी म्हणून
- **नमुन्याचा गुणात्मक आउटपुट** जलद तपासणीसाठी
- **गती वाढीची गणना** कार्यक्षमतेतील सुधारणा मोजण्यासाठी

**पर्यावरणीय घटक:**
- `SLM_ALIAS` - लहान भाषा मॉडेल (डीफॉल्ट: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - मोठे भाषा मॉडेल (डीफॉल्ट: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - तुलना करण्यासाठी चाचणी प्रॉम्प्ट
- `COMPARE_RETRIES` - टिकावासाठी पुन्हा प्रयत्न (डीफॉल्ट: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - सेवा एंडपॉइंट ओव्हरराइड करा (सेट केले नसल्यास स्वयंचलितपणे शोधले जाते)

**हे कसे कार्य करते (अधिकृत SDK पॅटर्न):**
1. **FoundryLocalManager** Foundry Local सेवेला प्रारंभ करतो आणि व्यवस्थापित करतो
2. सेवा चालू नसेल तर ती स्वयंचलितपणे सुरू होते (मॅन्युअल सेटअपची गरज नाही)
3. मॉडेल्स अॅलियासमधून ठोस आयडीमध्ये स्वयंचलितपणे रूपांतरित होतात
4. हार्डवेअर-ऑप्टिमाइझ्ड प्रकार निवडले जातात (CUDA, NPU, किंवा CPU)
5. OpenAI-सुसंगत क्लायंट चॅट पूर्णता करतो
6. मेट्रिक्स कॅप्चर केले जातात: प्रतीक्षा वेळ, टोकन्स, आउटपुट गुणवत्ता
7. निकालांची तुलना करून गती वाढीचा गुणोत्तर काढला जातो

ही सूक्ष्म तुलना तुमच्या वापराच्या प्रकरणासाठी मोठ्या मॉडेलकडे वळणे कधी योग्य आहे हे ठरविण्यास मदत करते.

**SDK संदर्भ:** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- वर्कशॉप युटिल्स: ../samples/workshop_utils.py मधील अधिकृत पॅटर्न वापरतो

**मुख्य फायदे:**
- ✅ स्वयंचलित सेवा शोध आणि प्रारंभ
- ✅ सेवा चालू नसेल तर स्वयंचलितपणे सुरू होते
- ✅ अंगभूत मॉडेल रिझोल्यूशन आणि कॅशिंग
- ✅ हार्डवेअर ऑप्टिमायझेशन (CUDA/NPU/CPU)
- ✅ OpenAI SDK सुसंगतता
- ✅ पुन्हा प्रयत्नांसह मजबूत त्रुटी हाताळणी
- ✅ स्थानिक अनुमान (क्लाउड API ची गरज नाही)


## 🚨 पूर्वअट: Foundry Local चालू असणे आवश्यक आहे!

**हा नोटबुक चालवण्यापूर्वी**, Foundry Local सेवा सेटअप केलेली असावी:

### जलद प्रारंभ आदेश (टर्मिनलमध्ये चालवा):

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### पर्यायी मॉडेल्स (जर डीफॉल्ट उपलब्ध नसतील):

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **जर तुम्ही हे पायऱ्या वगळल्या**, तर खालील नोटबुक सेल्स चालवताना तुम्हाला `APIConnectionError` दिसेल.


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### स्पष्टीकरण: मुख्य आयात
वेळ व्यवस्थापन साधने आणि Foundry Local / OpenAI क्लायंट्स आणते, जे मॉडेल माहिती मिळवण्यासाठी आणि चॅट पूर्णता करण्यासाठी वापरले जातात.


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### स्पष्टीकरण: उपनामे आणि प्रॉम्प्ट सेटअप  
लहान आणि मोठ्या मॉडेलसाठी पर्यावरणानुसार सानुकूलित होणारे उपनाम परिभाषित करतो तसेच तुलना प्रॉम्प्ट देखील. वेगवेगळ्या मॉडेल कुटुंबे किंवा कार्यांसाठी प्रयोग करण्यासाठी पर्यावरणीय चल (env vars) समायोजित करा.


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 मेमरी-ऑप्टिमाइझ्ड कॉन्फिगरेशन

**हे नोटबुक डिफॉल्टने मेमरी-इफिशियंट मॉडेल्स वापरते:**
- `phi-4-mini` → ~4GB RAM (Foundry Local स्वयंचलितपणे CPU व्हेरिएंट निवडते)
- `qwen2.5-3b` → ~3GB RAM (7B ऐवजी ज्याला ~7GB+ लागतात)

**पोर्ट ऑटो-डिटेक्शन:**
- Foundry Local वेगवेगळ्या पोर्ट्स वापरू शकते (सामान्यतः 55769 किंवा 59959)
- खालील डायग्नोस्टिक सेल योग्य पोर्ट स्वयंचलितपणे शोधतो
- कोणत्याही प्रकारची मॅन्युअल कॉन्फिगरेशन आवश्यक नाही!

**जर तुमच्याकडे मर्यादित RAM (<8GB) असेल, तर आणखी छोटे मॉडेल्स वापरा:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### स्पष्टीकरण: कार्यान्वयन सहाय्यक (Foundry SDK पॅटर्न)
वर्कशॉप नमुन्यांमध्ये दस्तऐवजीकरण केलेल्या अधिकृत Foundry Local SDK पॅटर्नचा वापर:

**पद्धत:**
- **FoundryLocalManager** - Foundry Local सेवा सुरू करते आणि व्यवस्थापित करते
- **स्वयंचलित शोध** - एंडपॉइंट आपोआप शोधतो आणि सेवा जीवनचक्र हाताळतो
- **मॉडेल रिझोल्यूशन** - उपनामांना पूर्ण मॉडेल आयडीमध्ये रूपांतरित करते (उदा., phi-4-mini → phi-4-mini-instruct-cpu)
- **हार्डवेअर ऑप्टिमायझेशन** - उपलब्ध हार्डवेअरसाठी सर्वोत्तम प्रकार निवडतो (CUDA, NPU, किंवा CPU)
- **OpenAI क्लायंट** - OpenAI-सुसंगत API प्रवेशासाठी व्यवस्थापकाच्या एंडपॉइंटसह कॉन्फिगर केलेले

**सहनशीलता वैशिष्ट्ये:**
- एक्स्पोनेंशियल बॅकऑफ रीट्राय लॉजिक (पर्यावरणाद्वारे कॉन्फिगर करता येण्याजोगे)
- सेवा चालू नसल्यास आपोआप सुरू होते
- प्रारंभानंतर कनेक्शन सत्यापन
- तपशीलवार त्रुटी अहवालासह सौम्य त्रुटी हाताळणी
- वारंवार प्रारंभ टाळण्यासाठी मॉडेल कॅशिंग

**परिणाम संरचना:**
- विलंब मोजमाप (वॉल क्लॉक वेळ)
- टोकन वापर ट्रॅकिंग (जर उपलब्ध असेल)
- नमुना आउटपुट (वाचनीयतेसाठी संक्षिप्त)
- अपयशी विनंत्यांसाठी त्रुटी तपशील

या पॅटर्नमध्ये workshop_utils मॉड्यूलचा उपयोग केला जातो, जो अधिकृत SDK पॅटर्नचे अनुसरण करतो.

**SDK संदर्भ:**
- मुख्य रेपो: https://github.com/microsoft/Foundry-Local
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- वर्कशॉप युटिल्स: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### स्पष्टीकरण: प्री-फ्लाइट सेल्फ-टेस्ट
FoundryLocalManager वापरून दोन्ही मॉडेल्ससाठी हलकासा कनेक्टिव्हिटी चेक चालवतो. यामध्ये खालील गोष्टींची खात्री केली जाते:
- सेवा प्रवेशयोग्य आहे
- मॉडेल्स सुरू होऊ शकतात
- उपनाम वास्तविक मॉडेल आयडींशी जुळतात
- तुलना चालवण्यापूर्वी कनेक्शन स्थिर आहे

setup() फंक्शन workshop_utils मधील अधिकृत SDK पॅटर्न वापरते.


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ जलद उपाय: नोटबुकमधून स्थानिक Foundry सुरू करा (ऐच्छिक)

वरील निदानानुसार सेवा चालू नसल्याचे दिसल्यास, तुम्ही ती इथून सुरू करण्याचा प्रयत्न करू शकता:

**टीप:** हे Windows वर उत्तम प्रकारे कार्य करते. इतर प्लॅटफॉर्मवर, टर्मिनल कमांड वापरा.


### ⚠️ कनेक्शन त्रुटींचे निराकरण

जर तुम्हाला `APIConnectionError` दिसत असेल, तर Foundry Local सेवा चालू नसावी किंवा मॉडेल्स लोड केलेले नसावेत. खालील चरणांचा प्रयत्न करा:

**1. सेवा स्थिती तपासा:**
```bash
# In a terminal (not in notebook):
foundry service status
```

**2. सेवा सुरू करा (जर चालू नसेल):**
```bash
foundry service start
```

**3. आवश्यक मॉडेल्स लोड करा:**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. मॉडेल्स उपलब्ध आहेत का ते सत्यापित करा:**
```bash
foundry model ls
```

**सामान्य समस्या:**
- ❌ सेवा चालू नाही → `foundry service start` चालवा
- ❌ मॉडेल्स लोड केलेले नाहीत → `foundry model run <model-name>` चालवा
- ❌ पोर्ट संघर्ष → तपासा की दुसरी सेवा पोर्ट वापरत आहे का
- ❌ फायरवॉल अडथळा → स्थानिक कनेक्शन परवानगी दिली आहे याची खात्री करा

**जलद उपाय:** प्री-फ्लाइट तपासणीपूर्वी खालील डायग्नोस्टिक सेल चालवा.


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ पूर्वतयारी तपासणी: मॉडेल उपलब्धता

ही सेल तुलना चालवण्यापूर्वी कॉन्फिगर केलेल्या एंडपॉइंटवर दोन्ही मॉडेल्स उपलब्ध आहेत का ते तपासते.


### स्पष्टीकरण: रन तुलना आणि निकाल गोळा करा
दोन्ही उपनामांवर अधिकृत Foundry SDK पॅटर्न वापरून पुनरावृत्ती करते:
1. प्रत्येक मॉडेलला setup() सह प्रारंभ करा (FoundryLocalManager वापरते)
2. OpenAI-सुसंगत API सह अनुमान चालवा
3. विलंब, टोकन आणि नमुना आउटपुट कॅप्चर करा
4. तुलनात्मक विश्लेषणासह JSON सारांश तयार करा

हे session04/model_compare.py मधील वर्कशॉप नमुन्यांप्रमाणेच पॅटर्न अनुसरते.


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### निकालांचे विश्लेषण

**महत्त्वाचे मेट्रिक्स:**
- **प्रतिक्रियेचा वेळ (Latency)**: कमी असणे चांगले - जलद प्रतिसाद वेळ सूचित करते
- **टोकन्स (Tokens)**: जास्त throughput = अधिक टोकन्स प्रक्रिया केली जातात
- **मार्ग (Route)**: कोणता API endpoint वापरला गेला हे पुष्टी करते

**SLM आणि LLM कधी वापरावे:**
- **SLM (लहान भाषा मॉडेल)**: जलद प्रतिसाद, कमी संसाधन वापर, साध्या कामांसाठी चांगले
- **LLM (मोठे भाषा मॉडेल)**: उच्च गुणवत्ता, चांगले तर्कशक्ती, गुणवत्ता महत्त्वाची असेल तेव्हा वापरा

**पुढील पावले:**
1. वेगवेगळ्या प्रॉम्प्ट्स वापरून पाहा आणि तुलना कशी बदलते ते पहा
2. इतर मॉडेल जोड्या वापरून प्रयोग करा
3. Workshop router samples (Session 06) वापरून कार्याच्या जटिलतेनुसार हुशारीने मार्ग निवडा


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**अस्वीकरण**:  
हा दस्तऐवज [Co-op Translator](https://github.com/Azure/co-op-translator) या एआय भाषांतर सेवेचा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमजांकरिता किंवा चुकीच्या अर्थ लावण्याकरिता आम्ही जबाबदार राहणार नाही.
